In [2]:
from langchain_groq import ChatGroq

In [ ]:
#enter your groq api key here
GROQ_API_KEY=""
llm = ChatGroq(
temperature = 0.7, 
    groq_api_key=GROQ_API_KEY,
    model_name = "llama-3.3-70b-versatile"
)

In [4]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
loader = WebBaseLoader("https://careers.nike.com/software-engineer-iii-blue-yonder-itc/job/R-68432")
page_data = loader.load().pop().page_content
print(page_data)





















Software Engineer III - Blue Yonder , ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Pr

In [6]:
from pydantic import BaseModel, Field
from typing import Optional, List

In [7]:
class JobInfo(BaseModel): 
    role : Optional[str] = Field(..., description="The job title or position")
    experience : Optional[str] = Field(..., description="Required experience level or years")
    skills : Optional[List[str]] = Field(..., description="List of required skills or technologies")
    description : Optional[str] = Field(..., description="Concise summary of the job description")

In [8]:
from langchain_core.output_parsers import JsonOutputParser
parser = JsonOutputParser(pydantic_object=JobInfo)

In [12]:
from langchain.prompts import ChatPromptTemplate

job_parser_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a job data extraction specialist. Extract information from job postings into JSON with this exact structure:
\n {format_instructions}
Output ONLY valid JSON, no other text."""),
    
    ("user", "Extract job information from this content: {scraped_content}")
]).partial(format_instructions=parser.get_format_instructions())

In [13]:
extract_chain = job_parser_prompt | llm | parser

In [15]:
response = extract_chain.invoke(input={"scraped_content":page_data})
response

{'role': 'Software Engineer III - Blue Yonder , ITC',
 'experience': '5+ years',
 'skills': ['BlueYonder TMS',
  'Transportation planning',
  'Execution',
  'Carrier management',
  'Freight audit processes',
  'Integration patterns with WMS and other supply chain systems',
  'API based integrations',
  'Computer Science',
  'Engineering',
  'Supply Chain'],
 'description': 'We are looking for a BlueYonder TMS Solution Architect to take a leading role in the configuration and implementation of BlueYonder TMS modules across Nike’s global logistics network.'}

In [17]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [19]:
import chromadb
import uuid
client = chromadb.PersistentClient("vectorstore")
collection = client.get_or_create_collection(name="portfolio")
if not collection.count() : 
    for _ , row in df.iterrows() :
        collection.add(
        documents = [row["Techstack"]], 
            metadatas= [{"link":row["Links"]}] , 
            ids=[str(uuid.uuid4())]
        )

In [45]:
results = collection.query(query_texts=[*response["skills"]], n_results=2)
results


{'ids': [['b5d2a816-7264-4ccc-b63f-3e7580903ac0',
   '463a012d-f46d-4ecd-8c36-560e5522e5d6'],
  ['21dc576e-1728-493f-8d7b-7a6871e71f66',
   '34b545bc-3354-46cd-a0a4-8680cc4ea7df'],
  ['a214a76c-5958-48d9-a131-e84b89a5cb50',
   '09807caf-49c1-4a49-8ecb-e67d9ad8e8d5'],
  ['cc62a318-2d8e-4220-8b49-397108090883',
   'a7f16194-72b5-475f-8e7a-89519cac1cce'],
  ['0a9777a5-7c68-4d08-b213-a8b7fd362bce',
   '21dc576e-1728-493f-8d7b-7a6871e71f66'],
  ['0a9777a5-7c68-4d08-b213-a8b7fd362bce',
   '2519d131-088e-41ac-95e4-7408d9623078'],
  ['09807caf-49c1-4a49-8ecb-e67d9ad8e8d5',
   '2519d131-088e-41ac-95e4-7408d9623078'],
  ['21dc576e-1728-493f-8d7b-7a6871e71f66',
   '09807caf-49c1-4a49-8ecb-e67d9ad8e8d5'],
  ['21dc576e-1728-493f-8d7b-7a6871e71f66',
   '0a9777a5-7c68-4d08-b213-a8b7fd362bce'],
  ['ffd0441c-16a7-4862-b29a-4d8b5b39418b',
   '162b0107-4674-41e4-8faa-c04901970901']],
 'embeddings': None,
 'documents': [['Backend, Kotlin, Spring Boot', 'React, Node.js, MongoDB'],
  ['Machine Learning, Pyt

In [46]:
links = results["metadatas"]
documents = results["documents"]
skills_info = []
for link_list , document_list in zip(links,documents) : 
    for link , document in zip(link_list,document_list):
        skills_info.append({"skills":document,"portfolio_link":link.get("link")})
skills_info

[{'skills': 'Backend, Kotlin, Spring Boot',
  'portfolio_link': 'https://example.com/kotlin-backend-portfolio'},
 {'skills': 'React, Node.js, MongoDB',
  'portfolio_link': 'https://example.com/react-portfolio'},
 {'skills': 'Machine Learning, Python, TensorFlow',
  'portfolio_link': 'https://example.com/ml-python-portfolio'},
 {'skills': 'Frontend, TypeScript, Angular',
  'portfolio_link': 'https://example.com/typescript-frontend-portfolio'},
 {'skills': 'Android, Java, Room Persistence',
  'portfolio_link': 'https://example.com/android-portfolio'},
 {'skills': 'Magento, PHP, MySQL',
  'portfolio_link': 'https://example.com/magento-portfolio'},
 {'skills': 'Kotlin, Android, Firebase',
  'portfolio_link': 'https://example.com/kotlin-android-portfolio'},
 {'skills': 'iOS, Swift, Core Data',
  'portfolio_link': 'https://example.com/ios-portfolio'},
 {'skills': 'DevOps, Jenkins, Docker',
  'portfolio_link': 'https://example.com/devops-portfolio'},
 {'skills': 'Machine Learning, Python, Ten

In [47]:
cold_email_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are a specialized email copywriter for the employment agency "{agency_name}". Your task is to generate a single, ready-to-send cold email based on the provided data.

You will receive:
1. Job opportunity in this JSON format: {{'role': '', 'experience': '', 'skills': [], 'description': ''}}
2. Portfolio information in this JSON format: {{"skills": [], "portfolio": "link"}}

Follow these guidelines:
- **Tone & Format:** Write in a professional, persuasive, and slightly conversational tone. Format the output as a complete email with Subject, Body, and Signature, using plain text only (no markdown).
- **Email Structure:** Use a proven effective structure:
    1. **Personalized Opening:** Address the hiring manager directly and create a strong hook.
    2. **Introduce the Candidate:** Present the candidate's profile, seamlessly integrating the specific skills, experience, and project details.
3. **Portfolio Integration:** 
       - Prioritize portfolio links that directly match skills mentioned in the job description
       - You can also highlight relevant skills and portfolio links that are related to the job description but not explicitly mentioned, if they add value
4. **Value Proposition:** Clearly state how the candidate's background aligns with the hiring manager's needs.
    5. **Call to Action:** End with a clear request to schedule a call or meeting.
- **Output:** Generate ONLY the final email text with no preamble, explanations, or additional text.

    """), 
    ("user","""
    Agency Name: {agency_name}
    Our Representative: {representative_name}
    Email Language: {email_language}

    Job Opportunity Details (JSON):
    {job_json}

    Relevant Candidate Project Portfolio Information (JSON):
    {portfolio_info}

    Generate the cold email.
    
    """)
])

In [51]:
email_chain = cold_email_prompt | llm 
cold_email_generated =email_chain.invoke(input={
    "agency_name": "Tech Talent Inc",
    "representative_name": "John Smith", 
    "email_language": "English",
    "job_json": response,
    "portfolio_info": skills_info
})
print(cold_email_generated.content)

Subject: Expert Software Engineer for Blue Yonder TMS Solution Architecture

Dear Hiring Manager,

I hope this email finds you well. As a representative of Tech Talent Inc, I am excited to introduce you to a highly skilled Software Engineer with over 5 years of experience, perfectly suited for the Software Engineer III - Blue Yonder, ITC role at your esteemed organization. With a strong background in BlueYonder TMS, Transportation planning, Execution, Carrier management, Freight audit processes, and Integration patterns with WMS and other supply chain systems, our candidate is confident in taking a leading role in the configuration and implementation of BlueYonder TMS modules.

Our candidate's expertise in API-based integrations, Computer Science, Engineering, and Supply Chain makes them an ideal fit for your team. Although their portfolio primarily showcases projects in various technologies such as Backend development with Kotlin and Spring Boot (https://example.com/kotlin-backend-por